In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Get value counts for the ASK_AMT column
df['ASK_AMT'].value_counts()

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [3]:
# Apply log transformation to ASK_AMT
df['ASK_AMT_LOG'] = np.log(df['ASK_AMT'])

# Bin the log-transformed values into categories
df['ASK_AMT_BINNED'] = pd.cut(df['ASK_AMT_LOG'], bins=[0, 10, 12, 14, 16, np.inf], 
                               labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
# Drop the original ASK_AMT column
df = df.drop(columns=['ASK_AMT'])

In [4]:
# Drop unnecessary columns
df = df.drop(columns=['EIN', 'NAME'])

# 'APPLICATION_TYPE' and 'CLASSIFICATION' based on cutoff points.
application_counts = df['APPLICATION_TYPE'].value_counts()
app_types_to_replace = application_counts[application_counts < 528].index
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app_types_to_replace, 'Other')

class_counts = df['CLASSIFICATION'].value_counts()
class_to_replace = class_counts[class_counts < 1883].index
df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(class_to_replace, 'Other')


# Convert categorical data to one-hot encoding
df = pd.get_dummies(df)

# Split features and target arrays
X = df.drop(columns=['IS_SUCCESSFUL']).values
y = df['IS_SUCCESSFUL'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    # Input layer
    nn_model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))


    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_inits',
                                                    min_value=1,
                                                    max_value=10,
                                                    step=2), activation=activation))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                        min_value=1,
                                                        max_value=10,
                                                        step=2),
                                           activation=activation))
        
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return nn_model


In [13]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [15]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

In [17]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_inits': 9,
 'num_layers': 6,
 'units_0': 7,
 'units_1': 7,
 'units_2': 7,
 'units_3': 3,
 'units_4': 5,
 'units_5': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0038'}

In [19]:
# Clear session before loading the best model
tf.keras.backend.clear_session()

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Evaluate the best model
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


/opt/anaconda3/lib/python3.12/site-packages/keras_tuner/src/tuners/hyperband.py:435: UserWarning: Model 'sequential' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  model.build_from_config(
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 1 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(48, 9), Received: value.shape=(43, 9). Target variable: <KerasVariable shape=(48, 9), dtype=float32, path=sequential/dense/kernel>

List of objects that could not be loaded:
[<Dense name=dense, built=True>]